<a href="https://colab.research.google.com/github/espickle1/esmfold_colabfold/blob/main/ESMFold_ColabFold_second.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Colabfold running ESMFold

In [1]:
## Install necessary packages
%%time
version = "1"
model_name = "esmfold_v0.model" if version == "0" else "esmfold.model"

import os, time

if not os.path.isfile(model_name):
  # download esmfold params
  os.system("apt-get install aria2 -qq")
  os.system(f"aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/{model_name} &")

  if not os.path.isfile("finished_install"):
    # install libs
    print("installing libs...")
    os.system("pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol modelcif")
    os.system("pip install -q git+https://github.com/NVIDIA/dllogger.git")

    print("installing openfold...")
    # install openfold
    os.system(f"pip install -q git+https://github.com/sokrypton/openfold.git")

    print("installing esmfold...")
    # install esmfold
    os.system(f"pip install -q git+https://github.com/sokrypton/esm.git")
    os.system("touch finished_install")

  # wait for Params to finish downloading...
  while not os.path.isfile(model_name):
    time.sleep(5)
  if os.path.isfile(f"{model_name}.aria2"):
    print("downloading params...")
  while os.path.isfile(f"{model_name}.aria2"):
    time.sleep(5)

CPU times: user 32 µs, sys: 0 ns, total: 32 µs
Wall time: 35.5 µs


In [2]:
## Import dependencies
import torch
from jax.tree_util import tree_map
import gc

from string import ascii_uppercase, ascii_lowercase
import hashlib, re, os
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import softmax
import py3Dmol

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
## Parse quantitative data for output
def parse_output(output):
  # Parse PAE and pLDDT
  pae = (output["aligned_confidence_probs"][0] * np.arange(64)).mean(-1) * 31
  plddt = output["plddt"][0,:,1]

  # Parse contacts
  bins = np.append(0,np.linspace(2.3125,21.6875,63))
  sm_contacts = softmax(output["distogram_logits"],-1)[0]
  sm_contacts = sm_contacts[...,bins<8].sum(-1)

  # Create overall position matrix
  xyz = output["positions"][-1,0,:,1]
  mask = output["atom37_atom_exists"][0,:,1] == 1

  # Combine output data
  o = {"pae":pae[mask,:][:,mask],
       "plddt":plddt[mask],
       "sm_contacts":sm_contacts[mask,:][:,mask],
       "xyz":xyz[mask]}

  return o

## Get has for unique item ID
def get_hash(x): return hashlib.sha1(x.encode()).hexdigest()

In [4]:
## Load model
if "model" not in dir() or model_name != model_name_:
  if "model" in dir():
    # delete old model from memory
    del model
    gc.collect()
    if torch.cuda.is_available():
      torch.cuda.empty_cache()

  # Load the model directly
  model = torch.load(model_name)
  model.eval().cuda().requires_grad_(False)
  model_name_ = model_name

<ipython-input-4-12623ac4d558>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_name)


In [5]:
## Import as a function
def sequence_read(sequence_input, position, copies):
  # Read translated sequences and parse to clean up invalid charaacters
  sequence = sequence_input.loc[position]['Translation']
  sequence_clean = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
  sequence_clean = re.sub(":+",":",sequence)
  sequence_clean = re.sub("^[:]+","",sequence)
  sequence_clean = re.sub("[:]+$","",sequence)

  # Read meta data from the file
  meta = sequence_input.loc[position]['meta']

  # Join multimers into one sequence
  if copies == "" or copies <= 0: copies = 1
  sequence = ":".join([sequence] * copies)

  # Come up with sequence ID and overall predicted length
  ID = "No_" + str(position) + "_" + jobname+"_"+get_hash(sequence)[:5]
  seqs = sequence.split(":")
  lengths = [len(s) for s in seqs]
  length = sum(lengths)
  print("length",length)

  # Determine the type of multimer
  u_seqs = list(set(seqs))
  if len(seqs) == 1: mode = "mono"
  elif len(u_seqs) == 1: mode = "homo"
  else: mode = "hetero"

  return sequence_clean, meta, copies, ID, u_seqs, length, lengths

In [6]:
## Predict protein structure from input sequence
def prediction_block(sequence, ID, row_number):
  # Determine chunk size for speedup
  # Following scheme is used (700 cutoff)
  # A100: 312 for smaller, 192 for larger
  # L4: 256 for smaller, 164 for larger
  # T4: 128 for smaller, 64 for larger
  length = len(sequence)
  if length > 700:
    model.set_chunk_size(192)
  else:
    model.set_chunk_size(312)

  # Empty cuda cache
  torch.cuda.empty_cache()

  # Inference
  output = model.infer(
      sequence,
      num_recycles=num_recycles,
      chain_linker="X"*chain_linker,
      residue_index_offset=512
      )

  # Parse outputs into exportable format
  pdb_str = model.output_to_pdb(output)[0]
  output = tree_map(lambda x: x.cpu().numpy(), output)
  ptm = output["ptm"][0]
  plddt = output["plddt"][0,...,1].mean()
  O = parse_output(output)
  print(f'ptm: {ptm:.3f} plddt: {plddt:.3f}')

  # Save results into files
  prefix = f"{ID}_ptm{ptm:.3f}_r{num_recycles}_default"
  np.savetxt(f"{prefix}.pae.txt",O["pae"],"%.3f")
  with open(f"{prefix}.pdb","w") as out:
    out.write(pdb_str)

  # return pdb_str, prefix, O
  return

In [7]:
## Import settings: manual settings
alphabet_list = list(ascii_uppercase+ascii_lowercase)

num_recycles = 3
chain_linker = 25
multimer_n = 1
# row_number = 12

jobname = "dir_test"
jobname = re.sub(r'\W+', '', jobname)[:50]

input_directory = "/content/drive/MyDrive/ww_virome/esmfold_colab/sequences/"
output_directory = "/content/drive/MyDrive/ww_virome/esmfold_colab/structures"
os.chdir(output_directory)
file_path = f"{input_directory}big_merge_norovirus_translation_20.csv"
sequence_file = pd.read_csv(file_path)

In [14]:
'''sequence_clean, meta, copies, ID, u_seqs, length, lengths = sequence_read(
    sequence_file,
    row_number,
    copies=multimer_n
    )

print(f"Entry {row_number}: {meta}")

# pdb_str, prefix, O = prediction_block(sequence_clean, ID, row_number)
start_time = time.time()
prediction_block(sequence_clean, ID, row_number)
end_time = time.time()
print(f"Inference time: {end_time - start_time:.2f}s")'''

length 1088
ptm: 0.441 plddt: 47.891
Inference time for entry 8, AGS43817.1_Equine rotavirus A_RNA-dependent RNA polymerase: 202.88s
CPU times: user 3min 20s, sys: 3.61 s, total: 3min 24s
Wall time: 3min 23s


In [9]:
# Get inference for files with multiple entries

start_time_overall = time.time()

for index, row in sequence_file.iterrows():
  sequence_clean, meta, copies, ID, u_seqs, length, lengths = sequence_read(
      sequence_file,
      index,
      copies=multimer_n
      )

  print(f"Entry {index + 1}: {meta}")

  start_time_loop = time.time()
  prediction_block(sequence_clean, ID, row)
  end_time_loop = time.time()
  print(f"Inference time for {meta}: \n{end_time_loop - start_time_loop:.2f}s")

end_time_overall = time.time()
print(f"Overall time: {end_time_overall - start_time_overall:.2f}s")


length 258
Entry 1: YP_009555234.1_Norovirus GII_VP2
ptm: 0.279 plddt: 53.239
Inference time for YP_009555234.1_Norovirus GII_VP2: 
6.28s
Overall time: 6.28s
length 268
Entry 2: YP_009518843.1_Norovirus GII_VP2
ptm: 0.304 plddt: 53.290
Inference time for YP_009518843.1_Norovirus GII_VP2: 
5.15s
Overall time: 11.42s
length 211
Entry 3: AGX01093.1_Norovirus Hu/GI.2/Jingzhou/2013401/CHN_minor capsid protein VP2
ptm: 0.302 plddt: 51.930
Inference time for AGX01093.1_Norovirus Hu/GI.2/Jingzhou/2013401/CHN_minor capsid protein VP2: 
3.11s
Overall time: 14.53s
length 257
Entry 4: AII73775.1_Norovirus GII/Hu/JP/2010/GII.P7_GII.7/Musashimurayama/TAKAsanKimchi_VP2
ptm: 0.266 plddt: 50.836
Inference time for AII73775.1_Norovirus GII/Hu/JP/2010/GII.P7_GII.7/Musashimurayama/TAKAsanKimchi_VP2: 
4.80s
Overall time: 19.34s
length 258
Entry 5: CRL46968.1_Norovirus GII/Hu/NL/2014/GII.6/Groningen_capsid protein VP2
ptm: 0.258 plddt: 50.443
Inference time for CRL46968.1_Norovirus GII/Hu/NL/2014/GII.6/Gron